# Аугментация исходного набора данных

In [1]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import shutil

import pandas as pd
import numpy as np

In [2]:
# флаг использования Google drive как хранилища данных
USE_GOOGLE_DRIVE = False

#Корневая папка проекта
root_dir = ''

In [3]:
#Подключаем google drive, если код будет выполняться в Colab
if USE_GOOGLE_DRIVE:
    from google.colab import drive

    drive.mount('/content/drive')
    root_dir = '/content/drive/My Drive/Projects/EmoDetector'
    
    import sys
    sys.path.append(root_dir)
    
#if

In [4]:
#import importlib
#importlib.reload (utils)

import emo_utils as utils

In [5]:
#Получаем пути к директориям и файлам проекта

paths = utils.get_paths(root_dir)

print (paths)

{'data_dir': 'data', 'model_dir': 'model', 'train_data_dir': 'data/train', 'train_aug_data_dir': 'data/train_aug', 'val_data_dir': 'data/val', 'test_data_dir': 'data/test_kaggle', 'class_names_path': 'data/class_names.txt', 'best_checkpoint_path': 'model/emo_classificator_best'}


In [6]:
# Гиперпараметры
BATCH_SIZE = 32
IMAGE_SIZE = 299
VALIDATION_SPLIT = 0.2

#Флаг создания валидационной выборки
CREATE_VAL_DATA = False
#Флаг удаления валидационной выборки
REMOVE_VAL_DATA = False
#Флаг создания аугментированной валидационной выборки
CREATE_TRAIN_AUG_DATA = True
#Флаг удаления обучающей выборки
REMOVE_TRAIN_AUG_DATA = True

In [7]:
#Подготовим генератор для аугментации картинок
image_gen = ImageDataGenerator(horizontal_flip=True,
                               zoom_range=0.2,
                               height_shift_range=0.2,
                               width_shift_range=0.2,
                               rotation_range=15)


In [10]:
#Соберем пути до изображений и индексы их эмоций

folder = []    
for i in os.walk(paths['train_data_dir']):
    folder.append(i)

data = {}
index = 0
for i in folder:
    for j in i[2]:       
        data [index] = {'path': i[0]+'/'+j,
                        'class': i[0].split('/')[-1],
                        'validation': 0}
        
        index = index + 1 
    #for

main_df = pd.DataFrame.from_dict(data, orient='index')

main_df

,path,class,validation
0,data/train/neutral/3994.jpg,neutral,0
1,data/train/neutral/389.jpg,neutral,0
2,data/train/neutral/2135.jpg,neutral,0
3,data/train/neutral/5649.jpg,neutral,0
4,data/train/neutral/691.jpg,neutral,0
...,...,...,...
50041,data/train/surprise/1340.jpg,surprise,0
50042,data/train/surprise/4481.jpg,surprise,0
50043,data/train/surprise/4143.jpg,surprise,0
50044,data/train/surprise/1830.jpg,surprise,0


In [11]:
#Соберем список классов

classes_names = list(set(main_df['class'].tolist()))

classes_names

['uncertain',
 'disgust',
 'happy',
 'surprise',
 'neutral',
 'fear',
 'sad',
 'anger',
 'contempt']

In [12]:
#Создаем директорию для валидационной и обучающей выборок

if REMOVE_VAL_DATA:
    if os.path.exists(paths['val_data_dir']):
        shutil.rmtree(paths['val_data_dir'], ignore_errors=True)
        
if CREATE_VAL_DATA:
    os.makedirs(paths['val_data_dir'], exist_ok=True)
    
if REMOVE_TRAIN_AUG_DATA:
    if os.path.exists(paths['train_aug_data_dir']):
        shutil.rmtree(paths['train_aug_data_dir'], ignore_errors=True)
        
if CREATE_TRAIN_AUG_DATA:
    os.makedirs(paths['train_aug_data_dir'], exist_ok=True)


In [13]:
#Отделим картинки для валидации от картинок для обучения

for class_name in classes_names:

    #выберем все картинки класса
    df = main_df[main_df['class'] == class_name]

    if CREATE_VAL_DATA:
        val_dir = paths['val_data_dir'] +'/'+class_name
        os.makedirs(val_dir, exist_ok=True)
    #if
    
    if CREATE_TRAIN_AUG_DATA:
        train_dir = paths['train_aug_data_dir'] +'/'+class_name
        os.makedirs(train_dir, exist_ok=True)
    #if

    #определим сколько картинок нужно для валидационной выборки
    val_cnt = int(VALIDATION_SPLIT * df.count()[0])
    cnt = 0
    img_in_batch = 0
    for row in df.iterrows():
        if cnt < val_cnt:
            if CREATE_VAL_DATA:
                shutil.copy(row[1]['path'], val_dir)
    
        else:
            if CREATE_TRAIN_AUG_DATA:
                shutil.copy(row[1]['path'], train_dir)
                
                img = image.load_img(row[1]['path'], target_size= (IMAGE_SIZE, IMAGE_SIZE))
                img = image.img_to_array(img)
                img = np.expand_dims(img, axis=0)
                
                if img_in_batch == 0:
                    images = img
                else:
                    images = np.append(images, img, axis=0)
                    
                img_in_batch = img_in_batch + 1
                
                #когда набрали пул картинок равный BATCH_SIZE, то генерируем аугментированные картинки 
                if img_in_batch == BATCH_SIZE:
                    train_generator = image_gen.flow (images, 
                                                      batch_size= BATCH_SIZE, 
                                                      save_to_dir= train_dir, 
                                                      save_format= 'jpeg',
                                                      save_prefix= 'a')
                    
                    next(train_generator)                    
                    
                    img_in_batch = 0 
                #if
                    
                
            #if
        
        
        cnt = cnt + 1

    #for

#for        
